In [ ]:
# #add missing packages to kernel
# import sys
# !{sys.executable} -m pip install pip install -r https://raw.githubusercontent.com/snowflakedb/snowflake-connector-python/v2.7.11/tested_requirements/requirements_38.reqs

In [94]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os as os
import json

In [99]:
def run_query(query):
    """
    this function takes a snowflake connection and executes a query with SQL commands in order to
    perform data transformations inside jupyter. 
    
    :param query: SQL commands to execute
    :returns: DataFrame with columns formatted 
    """
    import snowflake.connector

    # Assuming credentials are stored in same location as notebook
    CONFIG_LOCATION=os.path.join(os.getcwd(),'credentials.json')
    
    # load config
    CONFIG = json.loads(open(str(CONFIG_LOCATION)).read())
    
    # extract snowflake instance secrets from config
    SF_ACCOUNT    = CONFIG['secrets']['account']
    SF_USER       = CONFIG['secrets']['user']
    SF_WAREHOUSE  = CONFIG['secrets']['warehouse']
    SF_ROLE       = CONFIG['secrets']['role']
    SF_DATABASE   = CONFIG['secrets']['database']
    SF_SCHEMA     = CONFIG['secrets']['schema']
    SF_PASSWORD   = CONFIG['secrets']['password']
    
    #establish connection
    ctx = snowflake.connector.connect(
        account  = SF_ACCOUNT ,
        role     = SF_ROLE,
        user     = SF_USER,
        password = SF_PASSWORD,
        database = SF_DATABASE,
        schema   = SF_SCHEMA,
        warehouse= SF_WAREHOUSE
        )
    cs = ctx.cursor()
    try:
        cs.execute(query)
        result = cs.fetchall()
        columns=pd.DataFrame(cs.description)['name']#extracts column names
        df=pd.DataFrame(result, columns=columns)
        df=clean_columns(df)
        return df
    finally:
        cs.close()
    ctx.close()

In [14]:
def clean_columns(df):
    '''
    function to call on after DataFrame is created to format column names in lowercase 
    without spaces before querying.
    
    :param query: DataFrame
    :returns: DataFrame with columns formatted  
    '''
    columns=df.columns
    new_names=[]
    for c in columns:
        new_names.append(c
           .lower()
           .replace(' ', '_')
          )
    df.columns=new_names
    return df

In [101]:
#Ex: testing connection
q=( """
SELECT current_version();
""")
df=run_query(q)

name,CURRENT_VERSION()
0,6.27.0
